In [1]:
%%time
import pyodbc 
import pandas as pd
import numpy as np
import datetime
from functools import reduce
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
import timeit

warnings.filterwarnings('ignore')
%matplotlib inline
pd.set_option('display.max_columns', None)
#!pip install jupyter_contrib_nbextensions




ImportError: cannot import name 'remove_na'

In [6]:
%%time
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=LAPTOP-CU20T8L1;"
                      "Database=CAMPAIGN_DB2;"
                      "Trusted_Connection=yes;")


cursor = cnxn.cursor()
sql1 = pd.read_sql_query('SELECT TOP(1000) * FROM CAMPAIGN_TBL_FINAL',cnxn)
df1 = pd.DataFrame(sql1)

Wall time: 3.39 s


In [7]:
%%time
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=LAPTOP-CU20T8L1;"
                      "Database=CAMPAIGN_DB;"
                      "Trusted_Connection=yes;")


cursor = cnxn.cursor()
sql2 = pd.read_sql_query('SELECT TOP(1000) * FROM CAMPAIGN_TBL_FINAL',cnxn)
df2 = pd.DataFrame(sql2)

Wall time: 690 ms


In [8]:
%%time
df = pd.concat([df1,df2])
df.drop_duplicates() #dropping duplicates if any

Wall time: 232 ms


In [9]:
df.shape

(2000, 29)

In [10]:
%%time
df['Bet Time'] = pd.to_datetime(df['Bet Time'])
df['Registered Date'] = pd.to_datetime(df['Registered Date'])
df['Last Login'] = pd.to_datetime(df['Last Login'])

df['Transaction Date'] = df['Bet Time'].dt.date

 
last_betting_date = df.groupby(['Player ID'])['Bet Time'].agg(['max']).reset_index().rename(columns={"max":"Last Betting Date"})

bet_df = df.groupby('Player ID').agg({
                                        "Stake": {"total_stake": "sum","bets": "count"},
                                     }).reset_index()
bet_df.columns = ['Player ID','Total Stake','Total Bets']



dataframes = [bet_df,last_betting_date]
final_temp_df = reduce(lambda left, right: pd.merge(left, right, on='Player ID', how='left'), dataframes)

final_temp_df['days_since_last_bet'] = (final_temp_df['Last Betting Date'].max() - final_temp_df['Last Betting Date']).astype('timedelta64[D]')


SpecificationError: nested renamer is not supported

# RFM MODELLING

Recency (R): Who have purchased recently? Number of days since last purchase (least recency)

Frequency (F): Who has purchased frequently? It means the total number of purchases. ( high frequency)

Monetary Value(M): Who have high purchase amount? It means the total money customer spent (high monetary value)

In [39]:
# from crm_rfm_modeling import rfm
# from crm_rfm_modeling.rfm import RFM

,Player ID,days_since_last_login,Total Bets,Total Stake
0,0111555214,154.0,749,1805.40
1,02981pleeng,0.0,26613,155157.92
2,0619393970,59.0,572,1660.40
3,0621620658,254.0,50,4902.18
4,0624420637,257.0,7,670.00
5,0641306590,26.0,1159,3055.98
6,0641976840,0.0,2867,4525.29
7,0806409528pop,109.0,436,508.00
8,0820295597,251.0,109,287.20
9,0824308073wan,106.0,157,166.00


In [48]:
%%time
rfm_df = final_temp_df[['Player ID', 'days_since_last_login','Total Bets','Total Stake']]
rfm_df.columns = ['CustomerID','Recency','Frequency','Monetary']

Wall time: 2 ms


In [49]:
rfm_df.head()

,CustomerID,Recency,Frequency,Monetary
0,0111555214,154.0,749,1805.40
1,02981pleeng,0.0,26613,155157.92
2,0619393970,59.0,572,1660.40
3,0621620658,254.0,50,4902.18
4,0624420637,257.0,7,670.00


In [50]:
CustomerID ='0619393970' # has  R59.0 F572 M1660.40 for checking

In [51]:
%%time
rfm_df['RecencyScore'] = pd.qcut(rfm_df['Recency'],5,labels=[5,4,3,2,1])
rfm_df['FrequencyScore'] = pd.qcut(rfm_df['Frequency'].rank(method="first"),5,labels=[1,2,3,4,5])
rfm_df['MonetaryScore'] = pd.qcut(rfm_df['Monetary'],5,labels=[1,2,3,4,5])
rfm_df.head()

Wall time: 472 ms


In [52]:
(rfm_df['RecencyScore'].astype(str) + 
 rfm_df['FrequencyScore'].astype(str) + 
 rfm_df['MonetaryScore'].astype(str)).head()

0    344
1    555
2    444
3    224
4    113
dtype: object

In [53]:
%%time
rfm_df["RFM_SCORE"] = rfm_df['RecencyScore'].astype(str) + rfm_df['FrequencyScore'].astype(str) + rfm_df['MonetaryScore'].astype(str)

Wall time: 192 ms


In [54]:
rfm_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,937.0,139.816435,103.316161,0.0,32.00,113.0,253.0,258.0
Frequency,937.0,2134.471718,6687.940670,1.0,23.00,153.0,884.0,82328.0
Monetary,937.0,18262.930907,63451.030555,0.0,174.69,690.0,6420.0,816617.5


In [55]:
rfm_df[rfm_df["RFM_SCORE"] == "555"].head()

,CustomerID,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
1,02981pleeng,0.0,26613,155157.92,5,5,5,555
35,2537fon,0.0,3178,217202.26,5,5,5,555
61,akira88,0.0,13861,24706.50,5,5,5,555
63,alain1313,0.0,19851,51617.94,5,5,5,555
72,angsu1608,2.0,18278,159441.00,5,5,5,555


In [56]:
rfm_df[rfm_df["RFM_SCORE"] == "111"].head()

,CustomerID,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
70,an0620511331,256.0,5,8.00,1,1,1,111
128,bangkok2222,258.0,6,100.00,1,1,1,111
150,beerstc28,256.0,1,4.08,1,1,1,111
309,hongyok1a,257.0,1,100.00,1,1,1,111
412,kk1984,256.0,1,1.00,1,1,1,111


In [57]:
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Can\'t Lose',
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

In [59]:
rfm_df['Segment'] = rfm_df['RecencyScore'].astype(str) + rfm_df['FrequencyScore'].astype(str)
rfm_df['Segment'] = rfm_df['Segment'].replace(seg_map, regex=True)

In [60]:
rfm_df[["Segment", "Recency","Frequency","Monetary"]].groupby("Segment").agg(["mean","count"])


Recency          Frequency            Monetary      
                           mean count         mean count          mean count
Segment                                                                     
About to Sleep       139.594595    74    17.486486    74    297.021081    74
At Risk              251.127660   141   336.829787   141   6436.459929   141
Can't Lose           246.166667    12  2281.833333    12   7880.246667    12
Champions              3.414966   147  9119.068027   147  66620.009524   147
Hibernating          252.715596   218    21.788991   218   1086.147477   218
Loyal Customers       89.331210   157  3564.191083   157  34263.688153   157
Need Attention       137.129630    54   148.851852    54   4269.459074    54
New Customers          8.222222     9     5.444444     9    427.144444     9
Potential Loyalists   38.130435    92   117.097826    92   4748.908043    92
Promising             51.787879    33     4.242424    33    232.353636    33

### Remark:
3 segments selected for evaluation are "Can't Lose", "Hibernating" and "Champions".

Number of customers for segments:

Can't Lose = 12, Hibernating = 218, Champions = 147

#### Can't Lose Segment;

The last shopping date of the customers is on average 246 days before.
Customers have made an average of 16 purchases.
Customers spent an average of £ 8356.

#### Hibernating Segment;

The last shopping date of the customers is 481 days before average.
Customers made an average of 1 purchases.
Customers spent an average of £ 438.

#### Champions Segment;

The last shopping date of the customers is 30 days before average.
Customers made an average of 19 purchases.
Customers spent an average of £ 10796.

#### Can't Lose Segment;

Customers in this segment have not recently made a purchase. For this reason, we need to prepare a discount and gift campaign for this segment. These customers made a large number of purchases when they made purchases before. However, recency values are lower than they should be. The campaign to be implemented for these customers should include both items purchased and recommendations based on previous activities. New and popular products associated with the products that they were interested in can also be included in this campaign. Situations that will cause these customers to stop buying need to be investigated.

#### Hibernating Segment;

Customers in this segment have not made a purchase for a long time. However, by offering discounts, they may be attracted to another purchase.

#### Champions Segment;

Customers in this segment are responsible for most of the revenue. Campaigns should be implemented to ensure the continuity of the shopping of these customers.